# Install and Import, Start Pyspark Session

In [1]:
#pip install pyspark

  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845519 sha256=139a09e7e8f01ba4797691767b33278924125f2a95688a4c94849e9eb4945cb1
  Stored in directory: C:\Users\mstan\AppData\Local\pip\Cache\wheels\e5\cc\9a\0c20ee0940a9e80053edfe2270daee438f36037e1ff041c0ec
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [176]:
import pyspark

In [177]:
import pandas as pd

In [178]:
pd.read_csv('data/basic_csv_data.csv')

,album,genre,listeners,streams,rating
0,roses,rap,351.0,1138.0,6.0
1,jupiter,rap,135.0,6518.0,99.0
2,fire,rock,642.0,2491.0,7.0
3,venom,rock,786.0,3714.0,15.0
4,sky,classical,124.0,1142.0,18.0
5,danger,classical,123.0,6548.0,99.0
6,ocean,classical,NaN,NaN,20.0
7,NaN,NaN,364.0,3145.0,17.0
8,NaN,NaN,420.0,NaN,NaN


In [179]:
from pyspark.sql import SparkSession

In [180]:
spark = SparkSession.builder.appName('practice').getOrCreate()

In [181]:
spark

# View Data in Pyspark Session

In [182]:
df_pyspark = spark.read.option('header','true').csv('data/basic_csv_data.csv', inferSchema = True)

df_pyspark.show()

+-------+---------+---------+-------+------+
|  album|    genre|listeners|streams|rating|
+-------+---------+---------+-------+------+
|  roses|      rap|      351|   1138|     6|
|jupiter|      rap|      135|   6518|    99|
|   fire|     rock|      642|   2491|     7|
|  venom|     rock|      786|   3714|    15|
|    sky|classical|      124|   1142|    18|
| danger|classical|      123|   6548|    99|
|  ocean|classical|     null|   null|    20|
|   null|     null|      364|   3145|    17|
|   null|     null|      420|   null|  null|
+-------+---------+---------+-------+------+



In [226]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [183]:
df_pyspark.printSchema()

root
 |-- album: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- listeners: integer (nullable = true)
 |-- streams: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [184]:
df_pyspark.columns

['album', 'genre', 'listeners', 'streams', 'rating']

In [185]:
df_pyspark.head(2)

[Row(album='roses', genre='rap', listeners=351, streams=1138, rating=6),
 Row(album='jupiter', genre='rap', listeners=135, streams=6518, rating=99)]

In [186]:
df_pyspark.select(['album','listeners']).show() # fyi slicing will not work, seems to work without brackets 
# TODO check documentation

+-------+---------+
|  album|listeners|
+-------+---------+
|  roses|      351|
|jupiter|      135|
|   fire|      642|
|  venom|      786|
|    sky|      124|
| danger|      123|
|  ocean|     null|
|   null|      364|
|   null|      420|
+-------+---------+



In [187]:
df_pyspark.dtypes

[('album', 'string'),
 ('genre', 'string'),
 ('listeners', 'int'),
 ('streams', 'int'),
 ('rating', 'int')]

In [188]:
df_pyspark.describe().show()

+-------+------+---------+-----------------+-----------------+------------------+
|summary| album|    genre|        listeners|          streams|            rating|
+-------+------+---------+-----------------+-----------------+------------------+
|  count|     7|        7|                8|                7|                 8|
|   mean|  null|     null|          368.125|           3528.0|            35.125|
| stddev|  null|     null|246.8831056535519|2262.824562355642|39.740003235315136|
|    min|danger|classical|              123|             1138|                 6|
|    max| venom|     rock|              786|             6548|                99|
+-------+------+---------+-----------------+-----------------+------------------+



# Data Preprocessing with Pyspark

In [189]:
df_pyspark.na.drop(how='any', thresh = 2).show() # any is the default of how, 
# thresh is the number of non null values to be required in a row

+-------+---------+---------+-------+------+
|  album|    genre|listeners|streams|rating|
+-------+---------+---------+-------+------+
|  roses|      rap|      351|   1138|     6|
|jupiter|      rap|      135|   6518|    99|
|   fire|     rock|      642|   2491|     7|
|  venom|     rock|      786|   3714|    15|
|    sky|classical|      124|   1142|    18|
| danger|classical|      123|   6548|    99|
|  ocean|classical|     null|   null|    20|
|   null|     null|      364|   3145|    17|
+-------+---------+---------+-------+------+



In [190]:
df_pyspark.na.drop(how='all').show()

+-------+---------+---------+-------+------+
|  album|    genre|listeners|streams|rating|
+-------+---------+---------+-------+------+
|  roses|      rap|      351|   1138|     6|
|jupiter|      rap|      135|   6518|    99|
|   fire|     rock|      642|   2491|     7|
|  venom|     rock|      786|   3714|    15|
|    sky|classical|      124|   1142|    18|
| danger|classical|      123|   6548|    99|
|  ocean|classical|     null|   null|    20|
|   null|     null|      364|   3145|    17|
|   null|     null|      420|   null|  null|
+-------+---------+---------+-------+------+



In [191]:
df_pyspark.na.drop(subset='rating').show()

+-------+---------+---------+-------+------+
|  album|    genre|listeners|streams|rating|
+-------+---------+---------+-------+------+
|  roses|      rap|      351|   1138|     6|
|jupiter|      rap|      135|   6518|    99|
|   fire|     rock|      642|   2491|     7|
|  venom|     rock|      786|   3714|    15|
|    sky|classical|      124|   1142|    18|
| danger|classical|      123|   6548|    99|
|  ocean|classical|     null|   null|    20|
|   null|     null|      364|   3145|    17|
+-------+---------+---------+-------+------+



In [192]:
# how to replace nans with the mean for each column

from pyspark.ml.feature import Imputer

In [193]:
imputer = Imputer(
    inputCols = ['listeners', 'streams', 'rating'],
    outputCols = ["{}_imputed".format(c) for c in ['listeners', 'streams', 'rating']],
    ).setStrategy('mean')

In [194]:
df_pyspark = imputer.fit(df_pyspark).transform(df_pyspark)

In [195]:
df_pyspark.show()

+-------+---------+---------+-------+------+-----------------+---------------+--------------+
|  album|    genre|listeners|streams|rating|listeners_imputed|streams_imputed|rating_imputed|
+-------+---------+---------+-------+------+-----------------+---------------+--------------+
|  roses|      rap|      351|   1138|     6|              351|           1138|             6|
|jupiter|      rap|      135|   6518|    99|              135|           6518|            99|
|   fire|     rock|      642|   2491|     7|              642|           2491|             7|
|  venom|     rock|      786|   3714|    15|              786|           3714|            15|
|    sky|classical|      124|   1142|    18|              124|           1142|            18|
| danger|classical|      123|   6548|    99|              123|           6548|            99|
|  ocean|classical|     null|   null|    20|              368|           3528|            20|
|   null|     null|      364|   3145|    17|              36

In [196]:
drop_cols_list = ['listeners','streams','rating']

for col in drop_cols_list:
    df_pyspark = df_pyspark.drop(col)
    
df_pyspark.show()

+-------+---------+-----------------+---------------+--------------+
|  album|    genre|listeners_imputed|streams_imputed|rating_imputed|
+-------+---------+-----------------+---------------+--------------+
|  roses|      rap|              351|           1138|             6|
|jupiter|      rap|              135|           6518|            99|
|   fire|     rock|              642|           2491|             7|
|  venom|     rock|              786|           3714|            15|
|    sky|classical|              124|           1142|            18|
| danger|classical|              123|           6548|            99|
|  ocean|classical|              368|           3528|            20|
|   null|     null|              364|           3145|            17|
|   null|     null|              420|           3528|            35|
+-------+---------+-----------------+---------------+--------------+



In [197]:
rename_cols_dict = {'listeners_imputed':'listeners',
                    'streams_imputed':'streams',
                    'rating_imputed':'rating'}

for col in rename_cols_dict:
    df_pyspark = df_pyspark.withColumnRenamed(col,rename_cols_dict[col])
    
df_pyspark.show()

+-------+---------+---------+-------+------+
|  album|    genre|listeners|streams|rating|
+-------+---------+---------+-------+------+
|  roses|      rap|      351|   1138|     6|
|jupiter|      rap|      135|   6518|    99|
|   fire|     rock|      642|   2491|     7|
|  venom|     rock|      786|   3714|    15|
|    sky|classical|      124|   1142|    18|
| danger|classical|      123|   6548|    99|
|  ocean|classical|      368|   3528|    20|
|   null|     null|      364|   3145|    17|
|   null|     null|      420|   3528|    35|
+-------+---------+---------+-------+------+



In [198]:
df_pyspark = df_pyspark.withColumn('streams_per_listener',df_pyspark['streams']/df_pyspark['listeners'])

# Filter Operations

In [199]:
df_pyspark.filter(df_pyspark['rating'] >= 20).show()

+-------+---------+---------+-------+------+--------------------+
|  album|    genre|listeners|streams|rating|streams_per_listener|
+-------+---------+---------+-------+------+--------------------+
|jupiter|      rap|      135|   6518|    99|   48.28148148148148|
| danger|classical|      123|   6548|    99|  53.235772357723576|
|  ocean|classical|      368|   3528|    20|    9.58695652173913|
|   null|     null|      420|   3528|    35|                 8.4|
+-------+---------+---------+-------+------+--------------------+



In [200]:
df_pyspark.filter((df_pyspark['rating'] >= 8) & (df_pyspark['listeners'] >= 140)).show()

+-----+---------+---------+-------+------+--------------------+
|album|    genre|listeners|streams|rating|streams_per_listener|
+-----+---------+---------+-------+------+--------------------+
|venom|     rock|      786|   3714|    15|  4.7251908396946565|
|ocean|classical|      368|   3528|    20|    9.58695652173913|
| null|     null|      364|   3145|    17|    8.64010989010989|
| null|     null|      420|   3528|    35|                 8.4|
+-----+---------+---------+-------+------+--------------------+



In [201]:
df_pyspark.filter((df_pyspark['rating'] >= 8) & (df_pyspark['listeners'] >= 140)).select(['album',
                                                                                          'listeners',
                                                                                          'rating']).show()

+-----+---------+------+
|album|listeners|rating|
+-----+---------+------+
|venom|      786|    15|
|ocean|      368|    20|
| null|      364|    17|
| null|      420|    35|
+-----+---------+------+



# Aggregate Functions

In [202]:
df_pyspark.groupBy('genre').sum().show()

+---------+--------------+------------+-----------+-------------------------+
|    genre|sum(listeners)|sum(streams)|sum(rating)|sum(streams_per_listener)|
+---------+--------------+------------+-----------+-------------------------+
|     null|           784|        6673|         52|        17.04010989010989|
|      rap|           486|        7656|        105|        51.52364672364672|
|     rock|          1428|        6205|         22|        8.605253144990606|
|classical|           615|       11218|        137|        72.03240629881755|
+---------+--------------+------------+-----------+-------------------------+



In [203]:
df_pyspark.groupBy('genre').mean().show()

+---------+--------------+------------------+------------------+-------------------------+
|    genre|avg(listeners)|      avg(streams)|       avg(rating)|avg(streams_per_listener)|
+---------+--------------+------------------+------------------+-------------------------+
|     null|         392.0|            3336.5|              26.0|        8.520054945054945|
|      rap|         243.0|            3828.0|              52.5|        25.76182336182336|
|     rock|         714.0|            3102.5|              11.0|        4.302626572495303|
|classical|         205.0|3739.3333333333335|45.666666666666664|        24.01080209960585|
+---------+--------------+------------------+------------------+-------------------------+



# Regression with PySpark

In [218]:
from pyspark.ml.feature import VectorAssembler

featureassembler = VectorAssembler(inputCols = ['listeners','streams','streams_per_listener'], outputCol = 'independent_features')

output = featureassembler.transform(df_pyspark)

output.show()

+-------+---------+---------+-------+------+--------------------+--------------------+
|  album|    genre|listeners|streams|rating|streams_per_listener|independent_features|
+-------+---------+---------+-------+------+--------------------+--------------------+
|  roses|      rap|      351|   1138|     6|   3.242165242165242|[351.0,1138.0,3.2...|
|jupiter|      rap|      135|   6518|    99|   48.28148148148148|[135.0,6518.0,48....|
|   fire|     rock|      642|   2491|     7|    3.88006230529595|[642.0,2491.0,3.8...|
|  venom|     rock|      786|   3714|    15|  4.7251908396946565|[786.0,3714.0,4.7...|
|    sky|classical|      124|   1142|    18|   9.209677419354838|[124.0,1142.0,9.2...|
| danger|classical|      123|   6548|    99|  53.235772357723576|[123.0,6548.0,53....|
|  ocean|classical|      368|   3528|    20|    9.58695652173913|[368.0,3528.0,9.5...|
|   null|     null|      364|   3145|    17|    8.64010989010989|[364.0,3145.0,8.6...|
|   null|     null|      420|   3528|    35

In [219]:
regression_data = output.select(['independent_features', 'rating'])

regression_data.show()

+--------------------+------+
|independent_features|rating|
+--------------------+------+
|[351.0,1138.0,3.2...|     6|
|[135.0,6518.0,48....|    99|
|[642.0,2491.0,3.8...|     7|
|[786.0,3714.0,4.7...|    15|
|[124.0,1142.0,9.2...|    18|
|[123.0,6548.0,53....|    99|
|[368.0,3528.0,9.5...|    20|
|[364.0,3145.0,8.6...|    17|
|  [420.0,3528.0,8.4]|    35|
+--------------------+------+



In [220]:
from pyspark.ml.regression import LinearRegression

train_data, test_data = regression_data.randomSplit([0.75, 0.25])

regressor = LinearRegression(featuresCol = 'independent_features', labelCol = 'rating')

In [221]:
regressor.fit(regression_data).coefficients

DenseVector([-0.0184, 0.006, 1.2081])

In [222]:
regressor.fit(regression_data).intercept

0.8341156035419507

In [223]:
pred_results = regressor.fit(train_data).evaluate(test_data)
pred_results.predictions.show()

+--------------------+------+------------------+
|independent_features|rating|        prediction|
+--------------------+------+------------------+
|[123.0,6548.0,53....|    99|106.54699934644592|
|[351.0,1138.0,3.2...|     6|6.0863308997239685|
|[642.0,2491.0,3.8...|     7| 10.45093473547274|
+--------------------+------+------------------+



In [224]:
pred_results.meanSquaredError

22.957867569331512

In [225]:
pred_results.meanAbsoluteError

3.6947549938808746